# Sandbox

In [34]:
import torch
import numpy as np
import torch.multiprocessing as mp
from time import sleep

In [45]:
def worker(proc_num, queue):
    for _ in range(6000):
        if _ % 500 == 0:
            print(f'in {proc_num} on iteration {_}')
        rng = np.random.default_rng()
        length = rng.integers(48)
        state = rng.random(size=(length, 6))
        probs = rng.random(size=(5,))
        value = rng.random(size=(1,))
        datum = (state, probs, value)
        queue.put(datum)
#         with lock:
#             if len(buffer) < buffer_size:
#                 buffer.append(datum)
#                 index.value += 1
#             else:
#                 buffer[index.value] = datum
#                 index.value += 1
#             if index.value >= buffer_size:
#                 index.value = 0
    return


def checker(queue, output, buffer_size=24000):
    buffer = []
    index = 0
    while len(buffer) < buffer_size:
        if queue.empty():
            sleep(1)
            continue
        buffer.append(queue.get())
        index += 1
    output.put(buffer)

In [46]:
if __name__ == '__main__':
    # TODO start processes, try adding 450 * 48 * (tensor, tensor, tensor) to shared list
    # and see if I run into similar issues...
    with mp.Manager() as manager:
        queue = manager.Queue()
        output = manager.Queue()
#         length = manager.Value('i', 0)
#         index = manager.Value('i', 0)
#         lock = manager.Lock()
        workers = []
        for i in range(4):
            w = mp.Process(target=worker, args=(i, queue))
            w.start()
            workers.append(w)
        w = mp.Process(target=checker, args=(queue, output))
        w.start()
        workers.append(w)
        for w in workers:
            w.join()
        print('all processes joined')
#         buffer = list(buffer)
#         index = index.value
        buffer = output.get()

in 0 on iteration 0
in 1 on iteration 0in 2 on iteration 0in 3 on iteration 0


in 0 on iteration 500
in 3 on iteration 500in 1 on iteration 500

in 2 on iteration 500
in 0 on iteration 1000
in 1 on iteration 1000
in 2 on iteration 1000
in 3 on iteration 1000
in 1 on iteration 1500
in 2 on iteration 1500
in 0 on iteration 1500
in 3 on iteration 1500
in 1 on iteration 2000
in 2 on iteration 2000
in 0 on iteration 2000
in 3 on iteration 2000
in 1 on iteration 2500
in 2 on iteration 2500
in 0 on iteration 2500
in 3 on iteration 2500
in 1 on iteration 3000
in 2 on iteration 3000in 0 on iteration 3000

in 3 on iteration 3000
in 1 on iteration 3500
in 0 on iteration 3500
in 2 on iteration 3500
in 3 on iteration 3500
in 1 on iteration 4000
in 0 on iteration 4000in 2 on iteration 4000

in 3 on iteration 4000
in 1 on iteration 4500
in 0 on iteration 4500
in 2 on iteration 4500
in 3 on iteration 4500
in 1 on iteration 5000
in 0 on iteration 5000
in 2 on iteration 5000
in 3 on iteration 5000
in 1

In [41]:
buffer[2][2]

array([0.54937819])

In [47]:
[a[2] for a in buffer[:10]]

[array([0.76290919]),
 array([0.15618292]),
 array([0.79988231]),
 array([0.24052051]),
 array([0.95798373]),
 array([0.92766788]),
 array([0.49542138]),
 array([0.11634498]),
 array([0.28900658]),
 array([0.93548941])]

In [48]:
len(buffer)

24000